In [ ]:
import xarray as xr
import ocf_blosc2
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
nwp_data = xr.open_dataset("../../../mnt/disks/gcp_data/nwp/ecmwf/UK_v2.zarr")
meta_data = pd.read_csv("data_files/metadata.csv")
pv_data = xr.open_dataset("data_files/pv.netcdf", engine='h5netcdf')

In [ ]:
skip_ss_ids = ['8440', '16718', '8715', '17073', '9108', '9172', '10167', '10205', '10207', '10278', '26778', '26819', '10437', '10466', '26915', '10547', '26939', '26971', '10685', '10689', '2638', '2661', '2754', '2777', '2783', '2786', '2793', '2812', '2829', '2830', '2867', '2883', '2904', '2923', '2947', '2976', '2989', '2999', '3003', '3086', '3118', '3123', '3125', '3264', '3266', '3271', '3313', '3334', '3470', '3502', '11769', '11828', '11962', '3772', '11983', '3866', '3869', '4056', '4067', '4116', '4117', '4124', '4323', '4420', '20857', '4754', '13387', '13415', '5755', '5861', '5990', '6026', '6038', '6054', '14455', '6383', '6430', '6440', '6478', '6488', '6541', '6548', '6560', '14786', '6630', '6804', '6849', '6868', '6870', '6878', '6901', '6971', '7055', '7111', '7124', '7132', '7143', '7154', '7155', '7156', '7158', '7201', '7237', '7268', '7289', '7294', '7311', '7329', '7339', '7379', '7392', '7479', '7638', '7695', '7772', '15967', '7890', '16215', '7830']
hourly_pv_data = pv_data.sel(datetime=pv_data['datetime'].dt.minute == 0)
valid_ss_ids_data = [var for var in hourly_pv_data.data_vars if var not in skip_ss_ids]
pv_sites_id = np.random.choice(valid_ss_ids_data, 500, replace=False)
filtered_hourly_pv_data = hourly_pv_data[pv_sites_id]

In [ ]:
def get_36_hour_range(start_datetime, hours=36):
    end_datetime = start_datetime + pd.Timedelta(hours=hours - 1, minutes=59)
    return start_datetime, end_datetime

def select_non_overlapping_datetimes(datetimes, num_selections, min_gap_hours):
    selected_datetimes = []
    available_datetimes = list(datetimes)
    for _ in range(num_selections):
        if not available_datetimes:
            break
        random_datetime = np.random.choice(available_datetimes)
        selected_datetimes.append(random_datetime)
        available_datetimes = [dt for dt in available_datetimes if dt > random_datetime + pd.Timedelta(hours=min_gap_hours)]
    return selected_datetimes

In [ ]:
datetimes = pd.to_datetime(filtered_hourly_pv_data['datetime'].values)
data_dict = {'ss_id': [], 'pv_datetime': [], 'generation' : [], 'horizon':[]}
batch_size = 36
num_selections = 5000
min_gap_hours = 36

for ss_id in pv_sites_id:
    selected_datetimes = select_non_overlapping_datetimes(datetimes, num_selections, min_gap_hours)
    for start_datetime in selected_datetimes:
        start, end = get_36_hour_range(start_datetime, hours=batch_size)
        selected_data = hourly_pv_data.sel(datetime=slice(start, end))
        if len(selected_data['datetime']) < batch_size or selected_data[ss_id].isnull().any():
            continue
        hour_counter = 1
        batch_data = {'ss_id': [], 'pv_datetime': [], 'generation': [], 'horizon': []}
        for dt, power in zip(selected_data['datetime'].values, selected_data[ss_id].values):
            batch_data['ss_id'].append(int(ss_id))
            batch_data['pv_datetime'].append(dt)
            batch_data['generation'].append(power)
            batch_data['horizon'].append(hour_counter)
            hour_counter += 1
        if hour_counter - 1 == batch_size:
            for key in data_dict.keys():
                data_dict[key].extend(batch_data[key])

pv_df = pd.DataFrame(data_dict)
pv_df = pv_df.dropna(subset={'generation'})

In [ ]:
pv_sites_id = [int(id) for id in pv_sites_id]
pv_site_dict = {'ss_id':[], "lat":[], "long": [], 'tilt':[], 'orientation':[], 'kwp':[]}

for id in pv_sites_id:
    row = meta_data[meta_data['ss_id'] == id]
    if not row.empty:
        pv_site_dict['ss_id'].append(id)
        pv_site_dict['lat'].append(row['latitude_rounded'].values[0])
        pv_site_dict['long'].append(row['longitude_rounded'].values[0])
        pv_site_dict['tilt'].append(row['tilt'].values[0])
        pv_site_dict['orientation'].append(row['orientation'].values[0])
        pv_site_dict['kwp'].append(row['kwp'].values[0])

meta_site_df = pd.DataFrame.from_dict(pv_site_dict)
combined_df = pd.merge(pv_df, meta_site_df, on='ss_id', how='inner')
combined_df['pv_datetime'] = pd.to_datetime(combined_df['pv_datetime'])
combined_df['pv_date'] = combined_df['pv_datetime'].dt.date
combined_df['pv_hour'] = combined_df['pv_datetime'].dt.hour

In [ ]:
results = []
batch_size = 36
counter = 0

for i in tqdm(range(0, len(combined_df), batch_size), desc="Processing batches"):
    batch = combined_df.iloc[i:i + batch_size]
    if len(batch) < batch_size:
        continue
    initial_time = batch.iloc[0]['pv_datetime']
    lat = batch.iloc[0]['lat']
    lon = batch.iloc[0]['long']
    nwp_sel = nwp_data.sel(latitude=lat, method="nearest").sel(longitude=lon, method="nearest")
    init_time_sel = nwp_sel.sel(init_time=initial_time, method="ffill")
    if init_time_sel.init_time.size == 0:
        continue
    data_sel = init_time_sel.sel(step=slice(pd.Timedelta(hours=0), pd.Timedelta(hours=35)))
    data_df = data_sel.to_dataframe().reset_index()
    pivot_df = data_df.pivot_table(index=['init_time', 'step'], columns='variable', values='ECMWF_UK').reset_index()
    if len(pivot_df) < batch_size:
        continue
    for j in range(batch_size):
        pivot_df.loc[j, 'ss_id'] = batch.iloc[j]['ss_id']
        pivot_df.loc[j, 'pv_datetime'] = batch.iloc[j]['pv_datetime']
        pivot_df.loc[j, 'generation'] = batch.iloc[j]['generation']
        pivot_df.loc[j, 'horizon'] = batch.iloc[j]['horizon']
        pivot_df.loc[j, 'lat'] = lat
        pivot_df.loc[j, 'long'] = lon
        pivot_df.loc[j, 'tilt'] = batch.iloc[j]['tilt']
        pivot_df.loc[j, 'orientation'] = batch.iloc[j]['orientation']
        pivot_df.loc[j, 'kwp'] = batch.iloc[j]['kwp']
        pivot_df.loc[j, 'pv_hour'] = batch.iloc[j]['pv_hour']
    results.append(pivot_df)
    counter += 1

final_df = pd.concat(results, ignore_index=True)

In [ ]:
cumulative_vars = ['dlwrf', 'dswrf', 'duvrs', 'sr']

def cumulative_to_instantaneous(group):
    for var in cumulative_vars:
        group[f'{var}'] = group[var].diff().fillna(group[var])
    return group

final_df = final_df.groupby(['ss_id', 'init_time']).apply(cumulative_to_instantaneous).reset_index(drop=True)
final_df['normalize_generation'] = final_df['generation']/final_df['kwp']
final_df = final_df.rename(columns={'kwp': 'capacity'})

In [ ]:
rows_per_batch = 36
num_batches_to_keep = 100
rows_to_keep = rows_per_batch * num_batches_to_keep
train_data = final_df[:-rows_to_keep]
test_data = final_df[-rows_to_keep:]

In [ ]:
class LSTMForecaster(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super(LSTMForecaster, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
class SolarDataset(Dataset):
    def __init__(self, data, seq_length=36):
        self.data = data
        self.seq_length = seq_length
        weather_cols = ['dlwrf', 'dswrf', 'duvrs', 'hcc', 'lcc', 'mcc', 'sde', 'sr', 't2m', 'tcc', 'u10', 'u100', 'v10', 'v100']
        static_cols = ['capacity', 'lat', 'long', 'tilt', 'orientation']
        self.feature_cols = weather_cols + static_cols
        self.target_col = 'normalize_generation'
        self.scaler_X = StandardScaler()
        self.scaler_y = StandardScaler()
        self.X = self.scaler_X.fit_transform(data[self.feature_cols].values)
        self.y = self.scaler_y.fit_transform(data[[self.target_col]].values)
        
    def __len__(self):
        return len(self.data) - self.seq_length
    
    def __getitem__(self, idx):
        X_seq = self.X[idx:idx+self.seq_length]
        y_val = self.y[idx+self.seq_length]
        return torch.FloatTensor(X_seq), torch.FloatTensor(y_val)

In [ ]:
train_dataset = SolarDataset(train_data, seq_length=36)
test_dataset = SolarDataset(test_data, seq_length=36)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = len(train_dataset.feature_cols)
hidden_size = 128
num_layers = 3
output_size = 1
model = LSTMForecaster(input_size, hidden_size, num_layers, output_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 50
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
    
    train_losses.append(train_loss/len(train_loader))
    val_losses.append(val_loss/len(test_loader))
    
    if (epoch+1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('LSTM Training History')
plt.grid(True)
plt.show()

In [ ]:
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        predictions.extend(outputs.cpu().numpy())
        actuals.extend(y_batch.cpu().numpy())

predictions = test_dataset.scaler_y.inverse_transform(np.array(predictions))
actuals = test_dataset.scaler_y.inverse_transform(np.array(actuals))

mae = np.mean(np.abs(predictions - actuals))
print(f"Test MAE: {mae:.4f}")

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(actuals[:100], label='Actual', marker='o')
plt.plot(predictions[:100], label='Predicted', marker='x')
plt.xlabel('Sample')
plt.ylabel('Normalized Generation')
plt.title('LSTM: Actual vs Predicted Solar Generation')
plt.legend()
plt.grid(True)
plt.show()